## Sample Model

Based on the exploration, we try the following *as a starting point*.

> #### Delete the following columns
> - days_with_fog
> - direction_peak_wind_speed
> - direction_max_wind_speed
> - max_wind_speed
> - year_built
> - days_above_90F
> - days_above_110F
> - facility_type
> - site_eui (obviously, because target column)
> - For now also delete Year_Factor

> #### Impute missing values for energy_star_rating 
> Imputing is done by replacing nan by the mean <br>
> By second thoughts we do NOT impute as XGBoost infers missing values

> #### One-hot encode categorical values 
> - State_Factor
> - Building_Class

For the sample model, we use **random forest** and **xgboost**.

In [1]:
import sklearn
import xgboost as xgb
from evaluation import RMSE
from evaluation import helper_func
import pandas as pd
import gc
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error

#### Preparing and reading data

In [21]:
data = pd.read_csv("/Users/charlottefelius/documents/wids2022/WIDS/data/train.csv")
submission = pd.read_csv("/Users/charlottefelius/documents/wids2022/WIDS/data/test.csv")

In [23]:
len(submission)

9705

### Feature engineering

#### delete columns

In [3]:
reduced = pd.read_csv("/Users/charlottefelius/documents/wids2022/WIDS/data/train.csv")

to_delete = ["days_with_fog", "direction_peak_wind_speed", "direction_max_wind_speed", "max_wind_speed", "year_built", 
             "days_above_90F", "days_above_110F", "facility_type", "Year_Factor"]

for colname in to_delete:
    del reduced[colname]
    del submission[colname]
    
# collect garbage
gc.collect()


0

In [4]:
# print resulting columns
print(reduced.columns)
print(submission.columns)

Index(['State_Factor', 'building_class', 'floor_area', 'energy_star_rating',
       'ELEVATION', 'january_min_temp', 'january_avg_temp', 'january_max_temp',
       'february_min_temp', 'february_avg_temp', 'february_max_temp',
       'march_min_temp', 'march_avg_temp', 'march_max_temp', 'april_min_temp',
       'april_avg_temp', 'april_max_temp', 'may_min_temp', 'may_avg_temp',
       'may_max_temp', 'june_min_temp', 'june_avg_temp', 'june_max_temp',
       'july_min_temp', 'july_avg_temp', 'july_max_temp', 'august_min_temp',
       'august_avg_temp', 'august_max_temp', 'september_min_temp',
       'september_avg_temp', 'september_max_temp', 'october_min_temp',
       'october_avg_temp', 'october_max_temp', 'november_min_temp',
       'november_avg_temp', 'november_max_temp', 'december_min_temp',
       'december_avg_temp', 'december_max_temp', 'cooling_degree_days',
       'heating_degree_days', 'precipitation_inches', 'snowfall_inches',
       'snowdepth_inches', 'avg_temp', 'days_be

#### Impute variables (Skipped)

In [5]:
# impute with mean, ensures highest correlation
# reduced['energy_star_rating'] = reduced['energy_star_rating'].fillna(56.0)

In [6]:
# test which value enhances highest correlation still
# highest_corr = []

# for i in data['energy_star_rating'].unique():
#     d = pd.read_csv("/Users/charlottefelius/documents/wids2022/WIDS/data/train.csv")
#     d['energy_star_rating'] = d['energy_star_rating'].fillna(i)
#     correlations = helper_func.get_correlation(d, 'site_eui', 0)[1]
#     highest_corr.append((i, correlations[1]))

#### Onehot encoding

In [7]:
onehot = ["State_Factor", "building_class"]

def onehotter(dataframe, to_onehot):
    
    for i in to_onehot:
        ohe_df = pd.get_dummies(dataframe[i], prefix=i)

        # concat with original data
        dataframe = pd.concat([dataframe, ohe_df], axis=1).drop([i], axis=1)
        
    return dataframe

In [8]:
submission = onehotter(submission, onehot)
reduced = onehotter(reduced, onehot)

In [9]:
submission.columns

Index(['floor_area', 'energy_star_rating', 'ELEVATION', 'january_min_temp',
       'january_avg_temp', 'january_max_temp', 'february_min_temp',
       'february_avg_temp', 'february_max_temp', 'march_min_temp',
       'march_avg_temp', 'march_max_temp', 'april_min_temp', 'april_avg_temp',
       'april_max_temp', 'may_min_temp', 'may_avg_temp', 'may_max_temp',
       'june_min_temp', 'june_avg_temp', 'june_max_temp', 'july_min_temp',
       'july_avg_temp', 'july_max_temp', 'august_min_temp', 'august_avg_temp',
       'august_max_temp', 'september_min_temp', 'september_avg_temp',
       'september_max_temp', 'october_min_temp', 'october_avg_temp',
       'october_max_temp', 'november_min_temp', 'november_avg_temp',
       'november_max_temp', 'december_min_temp', 'december_avg_temp',
       'december_max_temp', 'cooling_degree_days', 'heating_degree_days',
       'precipitation_inches', 'snowfall_inches', 'snowdepth_inches',
       'avg_temp', 'days_below_30F', 'days_below_20F', 'days_

In [10]:
# save and pop submission id apart
Y_test_sub = submission["id"]
submission.pop("id")

0       75757
1       75758
2       75759
3       75760
4       75761
        ...  
9700    85457
9701    85458
9702    85459
9703    85460
9704    85461
Name: id, Length: 9705, dtype: int64

## Sample model

In [11]:
path = "/Users/charlottefelius/documents/WIDS2022/WIDS/data/train.csv"

#### Split into train and test set about  30 / 70 ratio

In [12]:
X = reduced
y = X['site_eui']
y_id = X['id']
X.pop('site_eui')
X.pop('id')
X.pop('State_Factor_State_6')

0        0
1        0
2        0
3        0
4        0
        ..
75752    0
75753    0
75754    0
75755    0
75756    0
Name: State_Factor_State_6, Length: 75757, dtype: uint8

In [13]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [14]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=123)

In [15]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.4,
                max_depth = 15, alpha = 1, n_estimators = 40)

In [16]:
# xg_reg.fit(X_train, y_train)
xg_reg.fit(X, y)

preds = xg_reg.predict(submission)
# preds = xg_reg.predict(X_test)

In [17]:
# rmse = np.sqrt(mean_squared_error(y_test, preds))
# print("RMSE: %f" % (rmse))

In [18]:
preds

array([ 84.76915,  68.39229, 164.82307, ..., 136.7066 ,  65.37906,
       136.15688], dtype=float32)

In [25]:
# Convert Preds to DataFrame 

result = pd.DataFrame(Y_test_sub)
result["site_eui"] = preds

In [31]:
# Write to CSV file

result.to_csv('/Users/charlottefelius/documents/WIDS2022/WIDS/results/first_attempt.csv', index=False, header=True)